# Tunnig da Rede Neural
#### Realizando um tunning na base de dados de carros usados.

> O objetivo dessa rede neural é analisar o preço de carros baseados em diversos atributos previsores.

### Importando as Bibliotecas

In [ ]:
# importando da biblioteca pandas do python
import pandas as pd
# importando da biblioteca keras do python
from keras.models import Sequential
from keras.layers import Dense
# importando da biblioteca sklearn do python
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

Using TensorFlow backend.


### Pré-processamento dos Dados

In [ ]:
# importando os dados para o dataframe
dataframe = pd.read_csv('/content/drive/My Drive/autos.csv', 
                        encoding = 'ISO-8859-1', sep = ',')

# removendo colunas não relevantes para a rede neural
dataframe = dataframe.drop('dateCrawled', axis = 1)
dataframe = dataframe.drop('dateCreated', axis = 1)
dataframe = dataframe.drop('nrOfPictures', axis = 1)
dataframe = dataframe.drop('postalCode', axis = 1)
dataframe = dataframe.drop('lastSeen', axis = 1)
dataframe = dataframe.drop('name', axis = 1)
dataframe = dataframe.drop('seller', axis = 1)
dataframe = dataframe.drop('offerType', axis = 1)

# removendo valores inconsistentes
dataframe = dataframe.loc[dataframe.price > 10]
dataframe = dataframe.loc[dataframe.price < 350000]

# criando um dicionário com a moda dos valores para substituir campos nulos
valores = {'vehicleType': 'limousine', 
          'gearbox': 'manuell', 
          'model': 'golf', 
          'fuelType': 'benzin', 
          'notRepairedDamage': 'nein'}
# atribuindo a moda dos dados aos campos nulos
dataframe = dataframe.fillna(value= valores)

# separando os atributos previsores dos atributos meta
previsores = dataframe.iloc[:, 1:13].values
classe = dataframe.iloc[:, 0].values

# criando o objeto 'labelencoder'
labelencoder_previsores = LabelEncoder()

# transformando atributos categóricos em numéricos
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 10] = labelencoder_previsores.fit_transform(previsores[:, 10])

# transformando atributos numéricos em variáveis do tipo dummy
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), 
                                                  [0,1,3,5,8,9,10])], 
                                  remainder='passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

### Criando a Rede Neural

In [ ]:
# configurando a rede neural
def criar_rede(loss):

  # criando o objeto regressor
  regressor = Sequential()
  # adicionando a primeira camada densa
  regressor.add(Dense(units = 158, activation = 'relu', 
                      kernel_initializer = 'random_uniform', 
                      input_dim = 316))
  # adicionando a segunda camada densa
  regressor.add(Dense(units = 158, activation = 'relu', 
                      kernel_initializer = 'random_uniform'))
  # adicionando a camada de saída
  regressor.add(Dense(units = 1, activation = 'linear'))
  # configurando os parâmetros da rede neural
  regressor.compile(loss=loss, optimizer='adam', 
                    metrics=['mean_absolute_error'])
  
  # função retorna o regressor
  return regressor

In [ ]:
# ajustando a rede neural para o treinamento
regressor = KerasRegressor(build_fn = criar_rede, 
                           epochs = 100, 
                           batch_size = 300, 
                           verbose = False)

In [ ]:
# criando um dicionário com os parâmetros de 'loss' a ser testado
parametros = {'loss': ['mean_squared_error', 'mean_absolute_error',
                       'mean_absolute_percentage_error', 
                       'mean_squared_logarithmic_error',
                       'squared_hinge']}

In [ ]:
# configurando os parâmetros do tunnig
grid_search = GridSearchCV(estimator = regressor,
                           param_grid = parametros,                           
                           cv = 10)

### Treinando a Rede Neural

In [ ]:
# treinando a rede neural com os dados tratados para descobrir quais são os 
# melhores parâmetros
grid_search = grid_search.fit(previsores, classe)

### Observando os Melhores Parâmetros Obtidos

In [ ]:
# visualizando os melhores parâmetros obtidos
melhores_parametros = grid_search.best_params_
print(melhores_parametros)

### Alguma Dúvida? Entre em Contato Comigo:

- [Me envie um e-mail](mailto:alysson.barbosa@ee.ufcg.edu.br);